In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
from spark_plot import mpl
mpl.set_defaults()

In [ ]:
spark = (SparkSession
    .builder
    .appName("nycflights-hist2d-dev")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .getOrCreate()
)

In [ ]:
from nycflights13 import flights as flights_pd

In [ ]:
flights = spark.createDataFrame(flights_pd)

In [ ]:
flights.persist()

In [ ]:
from spark_plot import utils
from pyspark.ml.feature import Bucketizer

In [ ]:
col_x = "sched_dep_time"
col_y = "sched_arr_time"
bins = 100

In [ ]:
data = flights[[col_x, col_y]]

In [ ]:
buckets_x = utils.spark_buckets(data, col_x, bins=bins)
buckets_y = utils.spark_buckets(data, col_y, bins=bins)

In [ ]:
bucketizer = Bucketizer(splits=buckets_x, inputCol=col_x, outputCol="bucket_x")
buckets_df = bucketizer.transform(data)

bucketizer = Bucketizer(splits=buckets_y, inputCol=col_y, outputCol="bucket_y")
buckets_df = bucketizer.transform(buckets_df)

In [ ]:
histogram = buckets_df.groupby("bucket_x", "bucket_y").agg(F.count(col_x).alias("count"))

In [ ]:
histogram = histogram.orderBy("bucket_x", "bucket_y", ascending=True)

In [ ]:
histogram.show()

In [ ]:
hist_pd = histogram.toPandas()

In [ ]:
hist_pd

In [ ]:
w = np.zeros((bins, bins))

In [ ]:
for index, row in hist_pd.iterrows():
    w[int(row['bucket_x']), int(row['bucket_y'])] = row["count"]

In [ ]:
w

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.pcolormesh(w)

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.pcolormesh(buckets_x, buckets_y, w.T)